# Behavioural Cloning Network 
##### this notebook contains code to drive a car autonomously through the Udacity Car Simulator 

In [1]:
import csv 
import cv2 
import numpy as np  
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

#Reading the CSV data file and saving the images for validation data

lines = [] 
with open('./udacity_data/driving_log.csv') as csvfile:
    next(csvfile)
    reader = csv.reader(csvfile)
    for line in reader: 
        lines.append(line)
        
print ('Creating data arrays...')
images = [] 
measurements = [] 
for line in lines: 
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = './udacity_data/IMG/' + filename
    image = cv2.imread(current_path)
    images.append(image)
    measurement = float(line[3]) 
    measurements.append(measurement)

X_val = np.array(images)
y_val = np.array(measurements)


print('Finished loading validation data.')

Creating data arrays...
Finished loading validation data.


In [2]:
# Reading the CSV data file and saving the images for train data 

lines = [] 
with open('./train_data/driving_log.csv') as csvfile:
    next(csvfile)
    reader = csv.reader(csvfile)
    for line in reader: 
        lines.append(line)
        
print ('Creating data arrays...')
images = [] 
measurements = [] 
for line in lines: 
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = './train_data/IMG/' + filename
    image = cv2.imread(current_path)
    images.append(image)
    measurement = float(line[3]) 
    measurements.append(measurement)


print('Finished loading first set of train data.')

extra_train_sets = 1

for i in range(extra_train_sets):
    lines = [] 
    path = './train_data' + str(i+1) + '/driving_log.csv'
    with open(path) as csvfile:
        next(csvfile)
        reader = csv.reader(csvfile)
        for line in reader: 
            lines.append(line)
    X_data = images 
    y_data = measurements
    print ('Creating data arrays...')
    

    path = './train_data' + str(i+1) + '/IMG/'
    for line in lines: 
        source_path = line[0]
        filename = source_path.split('/')[-1]
        current_path = path + filename
        image = cv2.imread(current_path)
        images.append(image)
        measurement = float(line[3]) 
        measurements.append(measurement)

    print('Finished loading ' + str(i+2) + 'th set of train data.')

print('Done')
X_data = np.array(images)
y_data = np.array(measurements)
print(X_data.shape)

Creating data arrays...
Finished loading first set of train data.
Creating data arrays...
Finished loading 2th set of train data.
Done
(11073, 160, 320, 3)


In [19]:
#image preprocessing steps 
import csv 
import cv2 
import numpy as np  
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import os

validation_samples = [] 

with open('./udacity_data/driving_log.csv') as csvfile:
    next(csvfile)
    reader = csv.reader(csvfile)
    for line in reader: 
        validation_samples.append(line)

train_samples = []
with open('./train_data/driving_log.csv') as csvfile:
    next(csvfile)
    reader = csv.reader(csvfile)
    for line in reader: 
        train_samples.append(line)        

with open('./train_data1/driving_log.csv') as csvfile:
    next(csvfile)
    reader = csv.reader(csvfile)
    for line in reader: 
        train_samples.append(line) 
        
print(len(train_samples))
def generator(samples, dir_name, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            images = []
            angles = []
            for batch_sample in batch_samples:
                index = 0 
                path = './'+ dir_name[index] + '/IMG/'+batch_sample[0].split('/')[-1]
                
                center_image = cv2.imread(path)
                if center_image is None: 
                    index += 1
                    path = './'+ dir_name[index] +'/IMG/'+batch_sample[0].split('/')[-1] 
                    center_image = cv2.imread(path)
                
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)
            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield X_train, y_train







11073


In [20]:
from keras.models import Sequential 
from keras.layers import Flatten, Dense, Lambda, Conv2D
from keras.backend import tf as ktf 

def NvidiaNet(): 
    
    #resizing & normalization lambda layers
    resize = Lambda(lambda img: ktf.image.resize_images(img, (66, 200)), input_shape=(160, 320, 3))
    normalize = Lambda(lambda x: (x / 255.0) - 0.5)
    
    model = Sequential() 
    model.add(resize) 
    model.add(normalize)
    
    # input: 3@66x200 conv ouput: 24@31x98
    model.add(Conv2D(24, kernel_size=(5,5), strides=(2, 2), padding='valid', activation='relu', kernel_initializer='he_normal', bias_initializer='zeros'))
    
    # conv output: 36@14x47
    model.add(Conv2D(36, kernel_size=(5,5), strides=(2, 2), padding='valid', activation='relu', kernel_initializer='he_normal', bias_initializer='zeros'))          
    
    # conv output: 48@5x22
    model.add(Conv2D(48, kernel_size=(5,5), strides=(2, 2), padding='valid', activation='relu', kernel_initializer='he_normal', bias_initializer='zeros'))    
    
    # conv output: 64@3x20
    model.add(Conv2D(64, kernel_size=(3,3), strides=(1, 1), padding='valid', activation='relu', kernel_initializer='he_normal', bias_initializer='zeros')) 
    
    # conv output: 64@1x18 
    model.add(Conv2D(64, kernel_size=(3,3), strides=(1, 1), padding='valid', activation='relu', kernel_initializer='he_normal', bias_initializer='zeros')) 
    
    # Fully connected layers
    model.add(Flatten())
    model.add(Dense(1164, kernel_initializer='he_normal', bias_initializer='zeros'))
    model.add(Dense(100, kernel_initializer='he_normal', bias_initializer='zeros'))
    model.add(Dense(50, kernel_initializer='he_normal', bias_initializer='zeros')) 
    model.add(Dense(10, kernel_initializer='he_normal', bias_initializer='zeros'))
    model.add(Dense(1, kernel_initializer='he_normal', bias_initializer='zeros'))
    
    model.compile(loss='mse', optimizer='adam',  metrics=['accuracy'])
    
    return model 


def SimpleNet():
    
    input_shape=(160,320,3)
    model = Sequential() 
    model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=input_shape))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Dense(1)) 

    model.compile(loss='mse', optimizer='adam',  metrics=['accuracy'])
    
    return model

    

In [21]:
import keras
print(keras.__version__)
model = NvidiaNet()


#model.fit(X_val, y_val, validation_split=0.2, shuffle=True, nb_epoch=7)
#model.fit(X_data, y_data, batch_size=256, shuffle=True, epochs=25, validation_data=(X_val, y_val))
# compile and train the model using the generator function
train_generator = generator(train_samples,['train_data', 'train_data1'], batch_size=256)
validation_generator = generator(validation_samples,['udacity_data'], batch_size=256)
model.fit_generator(train_generator, validation_data=validation_generator, validation_steps=8036/256 , epochs=7, steps_per_epoch=11073/256)
model.save('model.h5')

#./udacity_data/IMG/center_2016_12_01_13_30_48_287.jpg

2.0.6
Epoch 1/7
44/43 [==============================] - 20s - loss: 0.2233 - acc: 0.4012 - val_loss: 0.0201 - val_acc: 0.5427
Epoch 2/7
44/43 [==============================] - 20s - loss: 0.0086 - acc: 0.4164 - val_loss: 0.0187 - val_acc: 0.5427
Epoch 3/7
44/43 [==============================] - 20s - loss: 0.0072 - acc: 0.4164 - val_loss: 0.0188 - val_acc: 0.5427
Epoch 4/7
44/43 [==============================] - 20s - loss: 0.0062 - acc: 0.4164 - val_loss: 0.0214 - val_acc: 0.5427
Epoch 5/7
44/43 [==============================] - 20s - loss: 0.0055 - acc: 0.4164 - val_loss: 0.0187 - val_acc: 0.5427
Epoch 6/7
44/43 [==============================] - 20s - loss: 0.0050 - acc: 0.4164 - val_loss: 0.0171 - val_acc: 0.5427
Epoch 7/7
44/43 [==============================] - 20s - loss: 0.0047 - acc: 0.4164 - val_loss: 0.0164 - val_acc: 0.5427
